In [1]:
%load_ext autoreload
%autoreload 2

%load_ext blackcellmagic

In [2]:
import IPython.display as display
from IPython.display import HTML
display.display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import ray
ray.init(num_cpus=180)

2022-02-20 23:14:12,988	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8282


{'node_ip_address': '144.92.2.85',
 'raylet_ip_address': '144.92.2.85',
 'redis_address': '144.92.2.85:39897',
 'object_store_address': '/tmp/ray/session_2022-02-20_23-14-10_879781_283562/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-20_23-14-10_879781_283562/sockets/raylet',
 'webui_url': '127.0.0.1:8282',
 'session_dir': '/tmp/ray/session_2022-02-20_23-14-10_879781_283562',
 'metrics_export_port': 54721,
 'gcs_address': '144.92.2.85:45839',
 'node_id': '665d918e201e2bc43cf30ab0363289563c417ac4a0ade9237f6ebb67'}

In [5]:
import pyphi
from visualize_pyphi import utils
from visualize_pyphi import network_generator
from visualize_pyphi import fill_slots
import string
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import itertools
import toolz
import pandas as pd
from pyphi.models.subsystem import FlatCauseEffectStructure as sep
from visualize_pyphi.utils import ces2df
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'maximal-state-first'
IIT_VERSION = 'IIT_4.0_SMALL_PHI'
PARTITION_TYPE = 'TRI'
'''


pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE,pyphi.config.RELATION_PARTITION_TYPE,pyphi.config.RELATION_PARTITION_AGGREGATION,pyphi.config.SYSTEM_PARTITION_TYPE

('IIT_4.0_SMALL_PHI',
 'maximal-state-first',
 'TRI',
 'BI_CUT_ONE',
 'SUM',
 'TEMPORAL_DIRECTED_BI_CUT_ONE')

In [7]:
#Set weight degay (g) and determinism level (k)
g = 3
k = 6

In [8]:
net_name = "bigphi_final"

#Create weights for 7 nodes toroidal grid
n = 7
ws = [1/(u**g) for u in range(1,n)]
ws = [ws[i] for i in [0,1,1,2,2,3,3]]
ws = [w/sum(ws) for w in ws]

s = ws[0] #self (strong)
h = ws[1] #high
m = ws[3] #medium
w = ws[5] #weak
s,h,m,w

(0.7378309137489326,
 0.09222886421861658,
 0.027327070879590094,
 0.011528608027327072)

In [9]:
weights = np.array(
    [
        [s, h, m, w, w, m, h],  # A
        [h, s, h, m, w, w, m],  # B
        [m, h, s, h, m, w, w],  # C
        [w, m, h, s, h, m, w],  # D
        [w, w, m, h, s, h, m],  # E
        [m, w, w, m, h, s, h],  # F
        [h, m, w, w, m, h, s],  # G
        #A, B, C, D, E, F, G
    ]
)

node_labels = [string.ascii_uppercase[n] for n in range(len(weights))]
mech_func = ['l' for n in range(len(weights))] # "l" stands for LogFunc

network = network_generator.get_net(
    mech_func,
    weights,
    l=1,
    k=k,
    x0=.5,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=False,
)

state = (0,)*n

  0%|          | 0/128 [00:00<?, ?it/s]

## 3 nodes

In [57]:
n = 3
nodes = range(3)

subsystem3 = pyphi.Subsystem(network, state, nodes=nodes)
ces3 = pyphi.compute.ces(subsystem3)

# print(ces2df(ces3,subsystem3))

# generate all non conflicting distinctions sets
ncdss3 = list(pyphi.big_phi.all_nonconflicting_distinction_sets(ces3))

# compute all relations
ncdss3_rels = [pyphi.relations.relations(subsystem3, ncds) for ncds in ncdss3]

# pick compositional state based on existence
existences = [
    fill_slots.compute_existence(subsystem3, ncds, rels)
    for ncds, rels in zip(ncdss3, ncdss3_rels)
]
existences

max_ncds3 = ncdss3[existences.index(max(existences))]
max_ncds3_rels = ncdss3_rels[existences.index(max(existences))]

sia3 = pyphi.big_phi.sia(subsystem3, max_ncds3, max_ncds3_rels)

print(f"\n{n}-nodes big phi = {sia3.phi}")


Computing concepts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 59.23it/s]
                                                                                                                                                         

  0%|          | 0/77 [00:00<?, ?it/s]

  0%|          | 0/339 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/149 [00:00<?, ?it/s]

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state

3-nodes big phi = 45.623904251360884


## 4 nodes

In [30]:
%%time
n = 4
nodes = range(4)

subsystem4 = pyphi.Subsystem(network, state, nodes=nodes)
ces4 = pyphi.compute.ces(subsystem4)

# print(ces2df(ces4,subsystem4))

CPU times: user 1.12 s, sys: 40.7 ms, total: 1.16 s
Wall time: 1.12 s


In [32]:
# all_rels4 = pyphi.relations.relations(subsystem4, ces4)
# filename = f'relations/bigphi_final_for_Billy_{n}'
# all_rels4 = utils.compute_relations_by_k(subsystem4,ces4,filename)

In [45]:
# all_rels4 = utils.decompress_relations(subsystem4,ces4,f'{filename}*')
# len(all_rels4)

In [46]:
# generate all non conflicting distinctions sets
ncdss4 = list(pyphi.big_phi.all_nonconflicting_distinction_sets(ces4))
print(f"# of non-conflicting sets: {len(ncdss4)}")

# of non-conflicting sets: 128


In [53]:
existences = [fill_slots.estimate_existence(subsystem4,ces) for ces in ncdss4]

In [54]:
# pick compositional state based on existence
max_ncds4 = ncdss4[existences.index(max(existences))]
# ces2df(max_ncds4,subsystem4)

In [55]:
max_ncds4_rels = pyphi.relations.relations(subsystem4,max_ncds4)

  0%|          | 0/8299 [00:00<?, ?it/s]

In [56]:
sia4 = pyphi.big_phi.sia(subsystem4, max_ncds4, max_ncds4_rels)

print(f"\n{n}-nodes big phi = {sia4.phi}")

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state

4-nodes big phi = 50.61468472969473


## 5 nodes

In [59]:
n = 5
nodes = range(5)

subsystem5 = pyphi.Subsystem(network, state, nodes=nodes)
ces5 = pyphi.compute.ces(subsystem5)

# print(ces2df(ces5,subsystem5))


Computing concepts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:14<00:00,  1.38s/it]
                                                                                                                                                         

CPU times: user 14.6 s, sys: 313 ms, total: 14.9 s
Wall time: 14.6 s


In [60]:
# generate all non conflicting distinctions sets
ncdss5 = list(pyphi.big_phi.all_nonconflicting_distinction_sets(ces5))
print(f"# of non-conflicting sets: {len(ncdss5)}")

# of non-conflicting sets: 69696


In [76]:
# ncdss5_lens = set([len(s) for s in ncdss5])

# ncdss5_by_len = [[ces for ces in ncdss5 if len(ces)==l] for l in ncdss5_lens]

# ncdss5_max_len = ncdss5_by_len[-1]

In [74]:
existences = [fill_slots.estimate_existing_relations_number(subsystem5,ces) for ces in ncdss5]
max(existences)

876544

In [75]:
max_ncdss5_indices = [n for n,e in enumerate(existences) if e==max(existences)]
max_ncdss5_indices

[12020, 12029]

In [81]:
max_ncdss5 = [ncdss5[i] for i in max_ncdss5_indices]

In [83]:
[ces2df(ncdss5[i],subsystem5) for i in max_ncdss5_indices]

[   mechanism  state  cause  state       phi effect  state       phi
 1          A      0      A      0  0.694314      A      0  0.739839
 2          B      0      B      0  0.733409      B      0  0.749158
 3          D      0      D      0  0.733409      D      0  0.749158
 4          E      0      E      0  0.694314      E      0  0.739839
 5         AB     00     AB     00  0.028522     AB     00  0.056662
 6         AD     00     BC     00  0.010957    ABD    000  0.013185
 7         AE     00      C      0  0.010386    ACE    000  0.012766
 8         BE     00     CD     00  0.010957    BDE    000  0.013185
 9         DE     00     DE     00  0.028522     DE     00  0.056662
 10       ABC    000    ABC    000  0.033824    ABC    000  0.059513
 11       ABE    000     AD     00  0.010456   ABDE   0000  0.019106
 12       BCD    000    BCD    000  0.034153    BCD    000  0.067233
 13       CDE    000    CDE    000  0.033824    CDE    000  0.059513
 14      ABCD   0000   ABCD   0000

In [84]:
[fill_slots.count_purview_element_states_in_ces(ces,subsystem5) for ces in max_ncdss5]

[[[14, 0], [18, 0], [16, 0], [19, 0], [13, 0]],
 [[13, 0], [19, 0], [16, 0], [18, 0], [14, 0]]]

In [85]:
# pick compositional state based on existence
max_ncds5 = ncdss5[existences.index(max(existences))]
ces2df(max_ncds5,subsystem5)

,mechanism,state,cause,state,phi,effect,state,phi
1,A,0,A,0,0.694314,A,0,0.739839
2,B,0,B,0,0.733409,B,0,0.749158
3,D,0,D,0,0.733409,D,0,0.749158
4,E,0,E,0,0.694314,E,0,0.739839
5,AB,00,AB,00,0.028522,AB,00,0.056662
6,AD,00,BC,00,0.010957,ABD,000,0.013185
7,AE,00,C,0,0.010386,ACE,000,0.012766
8,BE,00,CD,00,0.010957,BDE,000,0.013185
9,DE,00,DE,00,0.028522,DE,00,0.056662
10,ABC,000,ABC,000,0.033824,ABC,000,0.059513


In [ ]:
path = 'relations/bigphi_final_for_Billy_5'
utils.compute_relations_by_k(subsystem5,max_ncds5,path)

  0%|          | 0/31 [00:00<?, ?it/s]

Computing k=2...


  0%|          | 0/367 [00:00<?, ?it/s]

[Parallel(n_jobs=160)]: Using backend LokyBackend with 160 concurrent workers.
[Parallel(n_jobs=160)]: Done   4 tasks      | elapsed:    4.5s
[Parallel(n_jobs=160)]: Done 122 out of 367 | elapsed:   11.0s remaining:   22.1s
[Parallel(n_jobs=160)]: Done 196 out of 367 | elapsed:   16.3s remaining:   14.2s
[Parallel(n_jobs=160)]: Done 270 out of 367 | elapsed:   20.8s remaining:    7.5s
[Parallel(n_jobs=160)]: Done 344 out of 367 | elapsed:   27.0s remaining:    1.8s


Computing k=3...


[Parallel(n_jobs=160)]: Done 367 out of 367 | elapsed:   29.5s finished


  0%|          | 0/2269 [00:00<?, ?it/s]

[Parallel(n_jobs=160)]: Using backend LokyBackend with 160 concurrent workers.
[Parallel(n_jobs=160)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=160)]: Done 256 tasks      | elapsed:   16.6s
[Parallel(n_jobs=160)]: Done 450 tasks      | elapsed:   37.7s
[Parallel(n_jobs=160)]: Done 648 tasks      | elapsed:  1.1min
[Parallel(n_jobs=160)]: Done 882 tasks      | elapsed:  1.6min
[Parallel(n_jobs=160)]: Done 1152 tasks      | elapsed:  2.1min
[Parallel(n_jobs=160)]: Done 1458 tasks      | elapsed:  2.8min
[Parallel(n_jobs=160)]: Done 1800 tasks      | elapsed:  3.5min


In [ ]:
max_ncds5_rels = utils.decompress_relations(subsystem5, max_ncds5, f"{path}*")

In [ ]:
sia5 = pyphi.big_phi.sia(subsystem5, max_ncds5, max_ncds5_rels)

print(f"\n{n}-nodes big phi = {sia5.phi}")